> Lưu ý. Tự triển khai mô hình sau đấy so sánh kết quả với mô hình chạy bằng scikit learn. Chú ý chỉnh sửa đường dẫn đến thư mục dữ liệu

In [3]:
import pandas as pd
import numpy as np

from sklearn.datasets import load_iris
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import coo_matrix # for sparse matrix

# Xác định ngôn ngữ 

In [4]:
# Load data
data = pd.read_csv('C:/HoangTu/Programing/Python/MachineLearningModelImplement/data/naivebayes/lang.csv')
X = data['Text']
y = data['language']
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(X)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=32)

In [ ]:
class MultinomialNaiveBayes:
    def fit(self, X: np.ndarray, y: np.ndarray):
        """
        Train the Naive Bayes model using the training data.
        
        Parameters:
        X : numpy array, shape (n_samples, n_features)
            Feature matrix of the training data.
        y : numpy array, shape (n_samples,)
            Array containing labels of the training data.

        Returns:
        None
        """
        # Initialize variables to store class information and feature counts
        self.classes = np.unique(y)  # Unique classes in the target variable y
        self.class_count = np.zeros(len(self.classes), dtype=np.float64)  # Count of each class

        self.feature_count = np.zeros((len(self.classes), X.shape[1]), dtype=np.float64)  # Count of each feature for each class
        self.feature_prob = np.zeros((len(self.classes), X.shape[1]), dtype=np.float64)  # Probability of each feature for each class

        # Calculate class counts and feature counts        
        for idx, c in enumerate(self.classes):
            X_c = X[y == c]  # Subset of X where y == c (instances of class c)
            self.class_count[idx] = X_c.shape[0]  # Number of instances of class c
            self.feature_count[idx, :] = X_c.sum(axis=0)  # Sum of all features for instances of class c

        # Calculate feature probabilities using Laplace smoothing
        self.feature_prob = (self.feature_count + 1) / (self.class_count[:, None] + X.shape[1])

        # Calculate class probabilities    
        self.class_prob = self.class_count / X.shape[0]  # Prior probability of each class
        return self

    def _predict(self, x):
        """
        Predict the class of a feature sample x.

        Parameters:
        x : numpy array, shape (n_features,)
            Feature sample to predict the class.

        Returns:
        predicted_class : object
            Predicted class for feature sample x.
        """
        posteriors = []
        for idx, c in enumerate(self.classes):
            prior = np.log(self.class_prob[idx])  # Logarithm of the prior probability of class c
            likelihood = np.sum(x * np.log(self.feature_prob[idx]))  # Log-likelihood of instance x belonging to class c
            posterior = prior + likelihood  # Posterior probability of class c given instance x
            posteriors.append(posterior)
        predicted_class = self.classes[np.argmax(posteriors)]  # Predicted class is the one with the highest posterior probability
        return predicted_class

    def predict(self, X: np.ndarray):
        """
        Predict the class of feature samples in matrix X.

        Parameters:
        X : numpy array, shape (n_samples, n_features)
            Feature matrix of samples to predict the class.

        Returns:
        predicted_classes : numpy array, shape (n_samples,)
            Array containing predicted classes for each feature sample in X.
        """
        predicted_classes = np.array([self._predict(x) for x in X])  # Predict class for each feature sample in X
        return predicted_classes


In [ ]:
model = MultinomialNB()
model.fit(x_train, y_train)
y_predict = model.predict(x_test)
accuracy_score(y_test, y_predict)

In [ ]:
model = MultinomialNaiveBayes()
model.fit(x_train,y_train)
y_predict = model.predict(x_test)
accuracy_score(y_test, y_predict)

# Iris data

Phân loại dữ liệu trên dự liệu hoa Iris:

In [ ]:
# Load data from sklearn
data = load_iris()
X = data.data
Y = data.target

In [ ]:
class GaussianNaiveBayes:
    def fit(self, X, y):
        """
        Fit Gaussian Naive Bayes model to training data.

        Parameters:
        X : array-like of shape (n_samples, n_features)
            Training data.
        y : array-like of shape (n_samples,)
            Target values.

        Returns:
        None
        """
        # Initialize parameters
        self.classes = np.unique(y)
        self.mean = np.zeros((len(self.classes), X.shape[1]), dtype=np.float64)
        self.var = np.zeros((len(self.classes), X.shape[1]), dtype=np.float64)
        self.priors = np.zeros(len(self.classes), dtype=np.float64)

        # Calculate mean, variance, and class priors for each class
        for idx, c in enumerate(self.classes):
            X_c = X[y == c]
            self.mean[idx, :] = X_c.mean(axis=0)
            self.var[idx, :] = X_c.var(axis=0)
            self.priors[idx] = X_c.shape[0] / X.shape[0]

    def _pdf(self, class_idx, x):
        """
        Compute the probability density function of Gaussian distribution.

        Parameters:
        class_idx : int
            Index of the class.
        x : array-like of shape (n_features,)
            Input data point.

        Returns:
        pdf : array-like of shape (n_features,)
            Probability density function values.
        """
        mean = self.mean[class_idx]
        var = self.var[class_idx]
        numerator = np.exp(- (x - mean)**2 / (2 * var))
        denominator = np.sqrt(2 * np.pi * var)
        return numerator / denominator

    def _predict(self, x):
        """
        Predict the class label for a single sample.

        Parameters:
        x : array-like of shape (n_features,)
            Input data point.

        Returns:
        predicted_class : int
            Predicted class label.
        """
        posteriors = []
        for idx, c in enumerate(self.classes):
            prior = np.log(self.priors[idx])
            posterior = np.sum(np.log(self._pdf(idx, x)))
            posterior = prior + posterior
            posteriors.append(posterior)
        return self.classes[np.argmax(posteriors)]

    def predict(self, X):
        """
        Predict the class labels for multiple samples.

        Parameters:
        X : array-like of shape (n_samples, n_features)
            Input data.

        Returns:
        predicted_classes : array-like of shape (n_samples,)
            Predicted class labels.
        """
        return np.array([self._predict(x) for x in X])

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=32)

In [ ]:
model = GaussianNB()
model.fit(x_train, y_train)
y_predict = model.predict(x_test)
accuracy_score(y_test, y_predict)

In [ ]:
model = GaussianNaiveBayes()
model.fit(x_train, y_train)
y_predict = model.predict(x_test)
accuracy_score(y_test, y_predict)

# Xác định thư spam

* Dữ liệu là tập con của bộ dữ liệu có tại link: [dataset](https://metatext.io/datasets/ling-spam-dataset). Tập dữ liệu này bao gồm tổng cộng 960 emails tiếng Anh, được tách thành tập training và test theo tỉ lệ 700:260, 50% trong mỗi tập là các spam emails.
* Bộ dữ liệu đã được tiền xử lý theo các bước:

* Loại bỏ stop words: Những từ xuất hiện thường xuyên như ‘and’, ‘the’, ‘of’, ... được loại bỏ.
* Lemmatization: Những từ có cùng ‘gốc’ được đưa về cùng loại. Ví dụ, ‘include’, ‘includes’, ‘included’ đều được đưa chung về ‘include’. Tất cả các từ cũng đã được
đưa về dạng ký tự thường (không phải HOA).
* Loại bỏ non-words: Số, dấu câu, ký tự ‘tabs’, ký tự ‘xuống dòng’ đã được loại bỏ.

* Bộ dữ liệu sau xử lý có trong link sau:
[ex6DataPrepared.zip](http://openclassroom.stanford.edu/MainFolder/courses/MachineLearning/exercises/ex6materials/) - Tải file ex6DataPrepared.zip

* Các tệp sau khi giải nén:
    * test-features.txt
    * test-labels.txt
    * train-features-50.txt
    * train-features-100.txt
    * train-features-400.txt
    * train-features.txt
    * train-labels-50.txt
    * train-labels-100.txt
    * train-labels-400.txt
    * train-labels.txt

Tương ứng với các file chứa dữ liệu của tập training và tập test. File train-features-50.txt chứ dữ liệu của tập training thu gọn với chỉ có tổng cộng 50 training emails. Mỗi file *labels*.txt chứa nhiều dòng, mỗi dòng là một ký tự 0 hoặc 1 thể hiện email là non-spam hoặc spam. Mỗi file *features*.txt chứa nhiều dòng, mỗi dòng có 3 số, ví dụ:

```
1 564 1
1 19 2
```


Trong đó số đầu tiên là chỉ số của email, bắt đầu từ 1; số thứ hai là thứ tự của từ trong từ điển (tổng cộng 2500 từ); số thứ ba là số lượng của từ đó trong email đang xét. Dòng đầu tiên nói rằng trong email thứ nhất, từ thứ 564 trong từ điển xuất hiện 1 lần. Cách lưu dữ liệu như thế này giúp tiết kiệm bộ nhớ vì 1 email thường không chứa hết tất cả các từ trong từ điển mà chỉ chứa một lượng nhỏ, ta chỉ cần lưu các giá trị khác không. Nếu ta biểu diễn feature vector của mỗi email là một vector hàng có độ dài bằng độ dài từ điển (2500) thì dòng thứ nhất nói rằng thành phần thứ 564 của vector này bằng 1. Tương tự, thành phần thứ 19 của vector này bằng 1. Nếu không xuất hiện, các thành phần khác được mặc định bằng 0.

> Tham khảo cách đọc dữ liệu từ file như sau: Lưu ý tự nghiên cứu cách chạy xử lý đến khi không tìm được cách mới xem cách đọc file

In [9]:
# data path and file name
path = 'C:/HoangTu/Programing/Python/MachineLearningModelImplement/data/naivebayes/email/'
train_data_fn = 'train-features.txt'
test_data_fn = 'test-features.txt'
train_label_fn = 'train-labels.txt'
test_label_fn = 'test-labels.txt'
n_words = 2500

def read_data(data_fn, label_fn):
    ## read label_fn
    with open(path + label_fn) as f:
        content = f.readlines()
    label = [int(x.strip()) for x in content]
    
    ## read data_fn
    with open(path + data_fn) as f:
        content = f.readlines()
    # remove '\n' at the end of each line
    content = [x.strip() for x in content]
    
    dat = np.zeros((len(content), 3), dtype = int)
    
    for i, line in enumerate(content):
        a = line.split(' ')
        dat[i, :] = np.array([int(a[0]), int(a[1]), int(a[2])])
    
    # remember to -1 at coordinate since we're in Python
    # check this: https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.coo_matrix.html
    # for more information about coo_matrix function
    data = coo_matrix((dat[:, 2], (dat[:, 0] - 1, dat[:, 1] - 1)), shape=(len(label), n_words))
    return (data, label)

(train_data, train_label) = read_data(train_data_fn, train_label_fn)
(test_data, test_label) = read_data(test_data_fn, test_label_fn)